In [ ]:
import tensorflow as tf
import keras
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization, ZeroPadding2D
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD, Adam
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
#------------------------------
#cpu - gpu configuration
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True)) 
keras.backend.set_session(sess)
#------------------------------
#variables
height = 48
width = 48
num_classes = 7 #angry, disgust, fear, happy, sad, surprise, neutral
batch_size = 64
epochs = 1000
#------------------------------
#read kaggle facial expression recognition challenge dataset (fer2013.csv)
#https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge

with open("../images/fer2013/fer2013.csv") as f:
    content = f.readlines()

lines = np.array(content)

num_of_instances = lines.size
print("number of instances: ",num_of_instances)
print("instance length: ",len(lines[1].split(",")[1].split(" ")))

#------------------------------
#initialize trainset and test set
x_train, y_train, x_test, y_test = [], [], [], []

#------------------------------
#transfer train and test set data
for i in range(1,num_of_instances):
    try:
        emotion, img, usage = lines[i].split(",")
          
        val = img.split(" ")
            
        pixels = np.array(val, 'float32')
        
        emotion = keras.utils.to_categorical(emotion, num_classes)
    
        if 'Training' in usage:
            y_train.append(emotion)
            x_train.append(pixels)
        elif 'PublicTest' in usage:
            y_test.append(emotion)
            x_test.append(pixels)
    except:
        print("",end="")

#------------------------------
#data transformation for train and test sets
x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')

x_train = x_train.reshape(x_train.shape[0], width, height, 1)
x_train = x_train.astype('float32')
x_test = x_test.reshape(x_test.shape[0], width, height, 1)
x_test = x_test.astype('float32')

#store mean and std dev for later
train_mean = np.mean(x_train)
train_std = np.std(x_train)
test_mean = np.mean(x_test)
test_std = np.std(x_test)

In [ ]:
from imutils import face_utils
import dlib
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import cv2
import skimage
from skimage import img_as_ubyte

# get coordinates of facial feature
def get_patch(c, size):
    top = 0 if c[1]-8 < 0 else c[1]-8
    bot = 47 if c[1]+8 > 47 else c[1]+8
    lef = 0 if c[0]-2 < 0 else c[0]-2
    rig = 47 if c[0]+size-2 > 47 else c[0]+size-2
    return top,bot,lef,rig

# return image with occlusion applied 
# occlusion applied randomly over estimated position of
# eyes or mouth if dlib cannot find landmarks
def noisy(noise_typ,image,d):
    
    if not d:
        d = {"mouth": np.array([16,38]), "both": np.array([11,18])}
        
    feature = random.choice([0,1])
    if feature == 0:
        top,bot,lef,rig = get_patch(d["both"],30)
    elif feature == 1:
        top,bot,lef,rig = get_patch(d["mouth"],25)
    modded = image.copy()
    row,col,ch = modded.shape
    if noise_typ == "s&p":
        noise = skimage.util.random_noise(modded,mode=noise_typ, amount = 1)
    else:
        noise = skimage.util.random_noise(modded,mode=noise_typ)
    modded[top:bot,lef:rig] = img_as_ubyte(noise[top:bot,lef:rig])
    return (modded.reshape(dim,dim,1))

#use DLIB to find facial landmarks, then apply occlusions
#code shows test set occluded but can be modified to use train set
dim = 48
p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)
W = np.squeeze(x_test_100, axis=3)
for x in range(0,len(W),1): # change step here to reduce % of occlusions
    gray = W[x].astype('uint8')
    rects = detector(gray, 1)
    features = {}
    # For each detected face, find the landmark.
    for (i, rect) in enumerate(rects):
        # Make the prediction and transfom it to numpy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        left_eye = shape[36] # position of left corner of left eye
        right_eye = shape[42] # position of right corner of right eye
        mouth = shape[48] # position of left corner of mouth
        features = {"mouth": mouth, "both":left_eye}
    gray = cv2.resize(gray, (dim,dim))
    gray = np.array(gray)
    gray = gray.reshape(dim, dim, 1)
    img = noisy("s&p",gray,features).astype('float32')
    x_test[x] = img

# apply z-score normalization after implementing occlusions
# without taking the occlusions into consideration
x_test -= test_mean
x_test /= test_std

In [ ]:
# LOAD MODEL AND TRAIN HERE